<a href="https://colab.research.google.com/github/rdkdaniel/Engineering--Arduino-Raspberry-Pi-etc/blob/main/Assembly_%26_Microprocessor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Key links: https://tonybaloney.github.io/posts/extending-python-with-assembly.html

https://specbranch.com/posts/python-and-asm/

**Python extensions in assembly language**

**Python is similar to assembly langua**ge

In [ ]:
Key links: https://tonybaloney.github.io/posts/extending-python-with-assembly.html

https://specbranch.com/posts/python-and-asm/

SyntaxError: ignored

# **Assembly, Disassembly and Emulation using Python**

In [1]:
#https://www.thepythoncode.com/article/arm-x86-64-assembly-disassembly-and-emulation-in-python

In [3]:
!pip3 install keystone-engine capstone unicorn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 5.4 MB/s 
     |████████████████████████████████| 2.1 MB 33.8 MB/s 
     |████████████████████████████████| 16.1 MB 39.3 MB/s 


# **Assembling ARM**

In [4]:
# We need to emulate ARM
from unicorn import Uc, UC_ARCH_ARM, UC_MODE_ARM, UcError
# for accessing the R0 and R1 registers
from unicorn.arm_const import UC_ARM_REG_R0, UC_ARM_REG_R1
# We need to assemble ARM code
from keystone import Ks, KS_ARCH_ARM, KS_MODE_ARM, KsError